In [1]:
import cv2
import os
import matplotlib.pyplot as plt
from tools import program
import tools.infer_rec as rec
import numpy as np
import paddle
import xlwt
import pdb

In [2]:
##设置
device = paddle.set_device('gpu:0')
logger = program.get_logger(name='root', log_file= './output/db_mv3/train.log')
#视频文件夹
path = './test3'
#输出excel文件
out_path = './result.xlsx'
#加载识别配置
rec_config = program.load_config('./configs/rec/rec_en_number_lite_train.yml')
#rec_config = program.load_config('./configs/rec/ch_PP-OCRv2_rec.yml')
#配置预训练识别模型
rec_config['Global']['pretrained_model']= "./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy"
#rec_config['Global']['pretrained_model']= "./output/rec/ch_PP-OCRv2_rec_slim_quant_train/best_accuracy"
#视频采集时间间隔, 默认两秒
time_step = 2

In [3]:
def write_xlsx(sheet, time_list,result_rec_1,result_rec_2):
    #列名
    sheet.write(0,0,'时间/秒')
    sheet.write(0,1,'当前体温/摄氏度')
    sheet.write(0,2,'传感器温度/摄氏度')
    for col, data in enumerate([time_list,result_rec_1,result_rec_2]):
        for row, d in enumerate(data):
            sheet.write(row+1,col,d,)

In [4]:
##因为数字区域不变，因此手动截出数字区域

# 可能区域1_1
pos_1_left_1 = 230
pos_1_right_1 = 405
pos_1_top_1 = 125
pos_1_bottom_1 = 190
# 可能区域1_2
pos_1_left_2 = 230
pos_1_right_2 = 410
pos_1_top_2 = 90
pos_1_bottom_2 = 160

#区域2_1
pos_2_left_1 = 140
pos_2_right_1 = 178
pos_2_top_1 = 282
pos_2_bottom_1 = 298

#区域2_2
pos_2_left_2 = 140
pos_2_right_2 = 178
pos_2_top_2 = 245
pos_2_bottom_2 = 265

#区域3
pos_3_left = 780
pos_3_right = 1150
pos_3_top = 170
pos_3_bottom = 250

workbook = xlwt.Workbook(encoding='utf-8')  

##获取视频列表
videos = sorted(os.listdir(path))
videos = [i for i in videos if not i.startswith('.')]
for idx, video in enumerate(videos):
    sheet = workbook.add_sheet(video,cell_overwrite_ok=True)
    ##读取视频并输出相关属性
    capture = cv2.VideoCapture(os.path.join(path,video))
    height = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame = capture.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = capture.get(cv2.CAP_PROP_FPS)
    time = frame/fps
    print('processing video ',idx,' height:',height,' width:',width,' frame:',frame,' fps:',fps, ' time:', str(time)+'s')
    ##获取视频数字区域二进制数据列表
    pos_1 = []
    pos_2 = []
    image_binary_1_1 = []
    image_binary_1_2 = []
    image_binary_2_1 = []
    image_binary_2_2 = []
    image_binary_3 = []
    time_list = []
    count = 0
    print('------------------------------提取视频温度目标区域------------------------------')
    #pbar = tqdm.tqdm(total=time)
    while(time_step*count<time):
        #pdb.set_trace()
        capture.set(cv2.CAP_PROP_POS_MSEC, time_step * 1000 * count)
        
        ret, frame = capture.read()
        
        if ret == False :
            break
        else:
            #截取待识别区域
            pos_1_1 = frame[pos_1_top_1:pos_1_bottom_1,pos_1_left_1:pos_1_right_1]
            pos_1_2 = frame[pos_1_top_2:pos_1_bottom_2,pos_1_left_2:pos_1_right_2]
            pos_2_1 = frame[pos_2_top_1:pos_2_bottom_1,pos_2_left_1:pos_2_right_1]
            pos_2_2 = frame[pos_2_top_2:pos_2_bottom_2,pos_2_left_2:pos_2_right_2]
            pos_3 = frame[pos_3_top:pos_3_bottom,pos_3_left:pos_3_right]
            image_binary_1_1.append(cv2.imencode('.jpg',pos_1_1)[1].tobytes())
            image_binary_1_2.append(cv2.imencode('.jpg',pos_1_2)[1].tobytes())
            image_binary_2_1.append(cv2.imencode('.jpg',pos_2_1)[1].tobytes())
            image_binary_2_2.append(cv2.imencode('.jpg',pos_2_2)[1].tobytes())
            image_binary_3.append(cv2.imencode('.jpg',pos_3)[1].tobytes())
            time_list.append(time_step * count)
        count += 1
        #pbar.update(time_step*count)
    #pbar.close()
    #
    ##识别两个区域数字
    print('------------------------------识别温度------------------------------')
    result_rec_1_1 = rec.main(rec_config, device, logger,  image_binary_1_1)
    result_rec_1_2 = rec.main(rec_config, device, logger,  image_binary_1_2)
    result_rec_2_1 = rec.main(rec_config, device, logger,  image_binary_2_1)
    result_rec_2_2 = rec.main(rec_config, device, logger,  image_binary_2_2)
    result_rec_3 = rec.main(rec_config, device, logger,  image_binary_3)
    #pdb.set_trace()
    #print(result_rec_1)
    result_rec_1_1_score = [i[0][1] if not np.isnan(i[0][1]) else '0' for i in result_rec_1_1]
    result_rec_1_2_score = [i[0][1] if not np.isnan(i[0][1]) else '0' for i in result_rec_1_2]
    result_rec_2_1_score = [i[0][1] if not np.isnan(i[0][1]) else '0' for i in result_rec_2_1]
    result_rec_2_2_score = [i[0][1] if not np.isnan(i[0][1]) else '0' for i in result_rec_2_2]
    result_rec_1 = [result_rec_1_1[i][0][0] if float(result_rec_1_1_score[i])>float(result_rec_1_2_score[i]) else result_rec_1_2[i][0][0] for i in range(len(result_rec_1_1))]
    result_rec_2 = [result_rec_2_1[i][0][0] if float(result_rec_2_1_score[i])>float(result_rec_2_2_score[i]) else result_rec_2_2[i][0][0] for i in range(len(result_rec_2_1))]
    result_rec_3 = [i[0][0] if float(i[0][1])>0.7 else '' for i in result_rec_3 ]
    
    ##写入结果
    write_xlsx(sheet, result_rec_3, result_rec_1, result_rec_2)
    
##保存   
workbook.save(out_path) 

processing video  0  height: 800.0  width: 1280.0  frame: 20694.0  fps: 10.346260242392669  time: 2000.143s
------------------------------提取视频温度目标区域------------------------------
------------------------------识别温度------------------------------
[2022/02/26 12:57:56] root INFO: load pretrained model from ['./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy']
[2022/02/26 12:58:26] root INFO: load pretrained model from ['./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy']
[2022/02/26 12:58:55] root INFO: load pretrained model from ['./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy']
[2022/02/26 13:00:13] root INFO: load pretrained model from ['./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy']
[2022/02/26 13:00:52] root INFO: load pretrained model from ['./output/rec/en_number_mobile_v2.0_rec_train/best_accuracy']
processing video  1  height: 800.0  width: 1280.0  frame: 19093.0  fps: 12.66353389068889  time: 1507.715s
------------------------------提取视频

In [4]:
videos = sorted(os.listdir(path))
videos = [i for i in videos if not i.startswith('.')]
capture = cv2.VideoCapture(os.path.join(path,videos[0]))
ret, frame = capture.read()

In [ ]:
plt.figure(dpi=280)
plt.imshow(frame[90:160,230:410])
plt.imshow(pos_2_1)